In [1]:
import pandas as pd
import numpy as np
import json

# Parsing
import yfinance as yf
from bs4 import BeautifulSoup
import requests

import warnings
warnings.filterwarnings("ignore")

# Macro Economics data from JSON

In [2]:
####################
### YahooFinance ###
####################
def get_yf_data(ticker):
    ticker = yf.Ticker(ticker)
    historical_data = ticker.history(period="max")  # data for max period
    return historical_data

### JSON openers ###
def extract_ceicdata_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        
    df = pd.DataFrame.from_records(data[0]['timePointList'])

    result_dict = {
        'data': df,
        'units': data[0]['seriesInfo']['displayUnit'],
        'name': data[0]['seriesInfo']['name']
    }
    return result_dict

def extract_investing_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        
    df = pd.DataFrame.from_records(data['data'])
    df.columns = ['date', 'value', 'misc']
    df['date'] = pd.to_datetime(df['date'],unit='ms')
    return df

###########################
### Basic preprocessing ###
###########################
def create_daterange_by_months(start_date:str, end_date:str):
    daterange = pd.date_range(start=start_date, end=end_date)
    dates = pd.DataFrame(data={'date':daterange})
    dates['date'] = dates['date'].dt.strftime('%Y-%m')
    dates = dates.drop_duplicates().reset_index(drop=True)
    return dates

def simple_processing(df:pd.DataFrame, column:str, method:str):    
    df = df[['date', 'value']]
    min_date, max_date = df['date'].min(), df['date'].max()

    ## Basic preprocessing of Inflation Dataset (rename columns + max value per month (if several)) 
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m')
    df['value'] = df['value'].astype(float)
    df = df.sort_values(by = ['date'], ascending=True)
    df = df.rename(columns = {'value': column})
    df = df.groupby(by = 'date')[column].mean().reset_index() # If several observations per month - take mean
    
    ## Filling missing months with T-1 (last month) value
    dates = create_daterange_by_months(start_date = str(min_date), end_date = str(max_date))
    df = dates.merge(df, how='left', on=['date'])                             # Missing months added and contain NaN value
    
    if method == 'ffill':
        df[column] = df[column].ffill()  
    else:
        df[f'{column}_'] = df[column].ffill()
        df[f'_{column}'] = df[column].bfill()
        df[column]  = df[column].fillna(df[[f'_{column}', f'{column}_']].mean(axis=1)) 
        df = df[['date', column]]
    return df

def show_stat(df):
    print(f'Data size: {df.shape[0]} x {df.shape[1]}')
    print(f'Min date: {df["date"].min()}')
    print(f'Max date: {df["date"].max()}')
    print(f'\nMissing values:\n{df.isnull().sum()}')

## External debt

In [3]:
data = extract_ceicdata_json('data/External_Debt.json')

external_debt = simple_processing(
    df        = data['data'],
    column    = 'external_debt',
    method    = 'ffill'
)

show_stat(external_debt)

external_debt.head()

Data size: 250 x 2
Min date: 2003-03
Min date: 2023-12

Missing values:
date             0
external_debt    0
dtype: int64


,date,external_debt
0,2003-03,153215.0
1,2003-04,153215.0
2,2003-05,153215.0
3,2003-06,158166.0
4,2003-07,158166.0


## Exports of goods and services

In [4]:
data = extract_ceicdata_json('data/Exports_of_Goods_and_Services.json')

exports       = simple_processing(
    df        = data['data'],
    column    = 'exports',
    method    = 'mean'
)

show_stat(exports)

exports.head()

Data size: 312 x 2
Min date: 1998-01
Min date: 2023-12

Missing values:
date       0
exports    0
dtype: int64


,date,exports
0,1998-01,5662.5
1,1998-02,5631.7
2,1998-03,6423.0
3,1998-04,5895.3
4,1998-05,5777.8


## Imports of goods and services

In [5]:
data = extract_ceicdata_json('data/Imports_of_Goods_and_Services.json')

imports       = simple_processing(
    df        = data['data'],
    column    = 'imports',
    method    = 'mean'
)

show_stat(imports)

imports.head()

Data size: 312 x 2
Min date: 1998-01
Min date: 2023-12

Missing values:
date       0
imports    0
dtype: int64


,date,imports
0,1998-01,4168.9
1,1998-02,4466.9
2,1998-03,4774.1
3,1998-04,4580.8
4,1998-05,4185.5


## GDP

In [6]:
data = extract_ceicdata_json('data/GDP_11-24.json')

gdp           = simple_processing(
    df        = data['data'],
    column    = 'GDP',
    method    = 'ffill'
)

show_stat(gdp)

gdp.head()

Data size: 154 x 2
Min date: 2011-03
Min date: 2023-12

Missing values:
date    0
GDP     0
dtype: int64


,date,GDP
0,2011-03,13024.799539
1,2011-04,13024.799539
2,2011-05,13024.799539
3,2011-06,14434.822306
4,2011-07,14434.822306


## Unemployment Rate

In [7]:
data = extract_investing_json("data/unemployment_rate.json")

unemplyment_rate  = simple_processing(
    df        = data,
    column    = 'unemplyment_rate',
    method    = 'ffill'
)

show_stat(unemplyment_rate)

unemplyment_rate.head()

Data size: 384 x 2
Min date: 1992-11
Min date: 2024-10

Missing values:
date                0
unemplyment_rate    0
dtype: int64


,date,unemplyment_rate
0,1992-11,4.8
1,1992-12,4.8
2,1993-01,4.8
3,1993-02,4.9
4,1993-03,5.1


## Interest Rate

In [8]:
data = extract_investing_json("data/interest_rate.json")

interest_rate  = simple_processing(
    df        = data,
    column    = 'interest_rate',
    method    = 'ffill'
)

show_stat(interest_rate)

interest_rate.head()

Data size: 257 x 2
Min date: 2003-06
Min date: 2024-10

Missing values:
date             0
interest_rate    0
dtype: int64


,date,interest_rate
0,2003-06,6.5
1,2003-07,6.5
2,2003-08,6.5
3,2003-09,6.5
4,2003-10,6.5


# Financial Data

In [9]:
# "GC=F"  - gold
# "NG=F"  - natural gas
# "CL=F"  - crude oil

# "IMOEX.ME" - Moscow Exchange Index
# "RUB=X" - USD exchange rate
# "EURRUB=X" - EUR exchange rate
# "RUBCNY=X" - CNY exchange rate

finance_data = dict()
for ticker in ["GC=F", "NG=F", "CL=F", "IMOEX.ME", "RUB=X", "EURRUB=X", "RUBCNY=X"]:
    df = get_yf_data(ticker).reset_index().rename(columns={'Date':'date'})
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m')
    df = df.groupby(by = 'date')['Close'].mean().reset_index()
    finance_data[ticker] = df[['date', 'Close']]

## Gold

In [10]:
gold_price = finance_data['GC=F'].rename(columns = {'Close': 'gold_price'})
show_stat(gold_price)
gold_price.head()

Data size: 294 x 2
Min date: 2000-08
Min date: 2025-01

Missing values:
date          0
gold_price    0
dtype: int64


,date,gold_price
0,2000-08,276.099991
1,2000-09,273.389996
2,2000-10,269.809090
3,2000-11,265.874997
4,2000-12,271.515005


## Gas

In [11]:
gas_price = finance_data['NG=F'].rename(columns = {'Close': 'gas_price'})
show_stat(gas_price)
gas_price.head()

Data size: 294 x 2
Min date: 2000-08
Min date: 2025-01

Missing values:
date         0
gas_price    0
dtype: int64


,date,gas_price
0,2000-08,4.792500
1,2000-09,5.146100
2,2000-10,5.081818
3,2000-11,5.741350
4,2000-12,8.664950


## Oil

In [12]:
oil_price = finance_data['CL=F'].rename(columns = {'Close': 'oil_price'})
show_stat(oil_price)
oil_price.head()

Data size: 294 x 2
Min date: 2000-08
Min date: 2025-01

Missing values:
date         0
oil_price    0
dtype: int64


,date,oil_price
0,2000-08,32.545714
1,2000-09,33.871000
2,2000-10,32.973182
3,2000-11,34.264500
4,2000-12,28.355000


## MOEX index

In [13]:
MOEX = finance_data["IMOEX.ME"].rename(columns = {'Close': 'MOEX'})
show_stat(MOEX)
MOEX.head()

Data size: 136 x 2
Min date: 2013-03
Min date: 2024-06

Missing values:
date    0
MOEX    0
dtype: int64


,date,MOEX
0,2013-03,1469.830635
1,2013-04,1384.005898
2,2013-05,1399.154274
3,2013-06,1317.945782
4,2013-07,1382.314782


## USD exchange rate

In [14]:
fx_usd_rub = finance_data["RUB=X"].rename(columns = {'Close': 'fx_usd_rub'})
show_stat(fx_usd_rub)
fx_usd_rub.head()

Data size: 251 x 2
Min date: 2003-12
Min date: 2025-01

Missing values:
date          0
fx_usd_rub    0
dtype: int64


,date,fx_usd_rub
0,2003-12,29.402609
1,2004-01,28.815714
2,2004-02,28.499000
3,2004-03,28.524644
4,2004-04,28.684000


## EUR exchange rate

In [15]:
fx_eur_rub = finance_data["EURRUB=X"].rename(columns = {'Close': 'fx_eur_rub'})
show_stat(fx_eur_rub)
fx_eur_rub.head()

Data size: 236 x 2
Min date: 2003-12
Min date: 2025-01

Missing values:
date          0
fx_eur_rub    0
dtype: int64


,date,fx_eur_rub
0,2003-12,36.187222
1,2004-01,36.347455
2,2004-02,36.083011
3,2004-04,34.990002
4,2004-07,35.070000


# Other collected Features

## Inflation (Target)

In [16]:
dict_months = {
    " Январь"  : "-01",
    " Февраль" : "-02",
    " Март"    : "-03",
    " Апрель"  : "-04",
    " Май"     : "-05",
    " Июнь"    : "-06",
    " Июль"    : "-07",
    " Август"  : "-08",
    " Сентябрь": "-09",
    " Октябрь" : "-10",
    " Ноябрь"  : "-11",
    " Декабрь" : "-12",
}
inflation = pd.read_csv('data/russia_inflation_monthly.csv')
inflation = inflation.drop([0]).drop(columns=[' Всего'])

frames = []
for column in list(inflation.columns)[1:]:
    tmp = inflation[['Год', column]]
    tmp['Date'] = tmp['Год'].astype(str) + dict_months[column]
    tmp = tmp[['Date', column]]
    tmp.columns = ['date', 'inflaton_rate']
    frames.append(tmp)
    
inflation = pd.concat(frames, ignore_index=True)
inflation = inflation.sort_values(by = ['date'], ignore_index=True)

show_stat(inflation)
inflation.head()

Data size: 396 x 2
Min date: 1992-01
Min date: 2024-12

Missing values:
date             0
inflaton_rate    3
dtype: int64


,date,inflaton_rate
0,1992-01,746.67
1,1992-02,1014.89
2,1992-03,1262.41
3,1992-04,914.10
4,1992-05,1001.73


## Consumer Price Index

In [17]:
CPI = extract_ceicdata_json("data/CPI.json")
CPI = CPI['data'][['value', 'date']]
CPI['date'] = pd.to_datetime(CPI['date']).dt.strftime('%Y-%m')
CPI = CPI[['date', 'value']]
CPI.columns = ['date', 'CPI']
CPI = CPI.sort_values(by = 'date', ascending = True)

show_stat(CPI)
display(CPI.head())

Data size: 165 x 2
Min date: 2011-01
Min date: 2024-09

Missing values:
date    0
CPI     0
dtype: int64


,date,CPI
164,2011-01,106.2
163,2011-02,107
162,2011-03,107.7
161,2011-04,108.1
160,2011-05,108.7


## Goods Prices

In [18]:
def rename_columns_data(df):
    dict_months = {
        "январь"  : "-01",
        "февраль" : "-02",
        "март"    : "-03",
        "апрель"  : "-04",
        "май"     : "-05",
        "июнь"    : "-06",
        "июль"    : "-07",
        "август"  : "-08",
        "сентябрь": "-09",
        "октябрь" : "-10",
        "ноябрь"  : "-11",
        "декабрь" : "-12",
    }
    
    new_columns = []
    for i in df.columns:
        m = i.split()[0]
        year = i.split()[1]
        if m in dict_months.keys():
           # print(m, year + dict_months[m])    
            new_columns.append(year + dict_months[m])
        else:
            new_columns.append(i)
    df.columns = new_columns
    return df

goods1 = pd.read_excel('data/consumer_prices/2000-2003.xlsx',sheet_name=0, skiprows = 1)
goods2 = pd.read_excel('data/consumer_prices/2004-2010.xlsx',sheet_name=0, skiprows = 1)
goods3 = pd.read_excel('data/consumer_prices/2011-2024.xlsx',sheet_name=0, skiprows = 1)

goods1 = rename_columns_data(goods1)
goods2 = rename_columns_data(goods2)
goods3 = rename_columns_data(goods3)

first = pd.merge(goods1, goods2, how='inner', on=['Unnamed: 0', 'Unnamed: 1'])
goods = pd.merge(first, goods3, how='inner', on=['Unnamed: 0', 'Unnamed: 1'])

goods_names = goods['Unnamed: 0'] .to_list()
goods = goods.drop(columns = ['Unnamed: 0', 'Unnamed: 1'])

goods = goods.T
goods.columns = goods_names


# Creating a dataset for three products included in the basic consumer basket in Russia without missing values. 
# This dataset consists of the prices of the following products per month: chicken eggs (10 pcs.), 
# granulated sugar (1 kg), and wheat flour (1 kg).

main_goods = goods[['Яйца куриные, 10 шт.', 'Сахар-песок, кг', 'Мука пшеничная, кг']]
main_goods = main_goods.reset_index()
main_goods.columns = ['date','chicken_eggs_10pcs','granulated_sugar_1kg', 'wheat_flour_1kg']

show_stat(main_goods)
main_goods.head()

Data size: 297 x 4
Min date: 2000-01
Min date: 2024-09

Missing values:
date                    0
chicken_eggs_10pcs      0
granulated_sugar_1kg    0
wheat_flour_1kg         0
dtype: int64


,date,chicken_eggs_10pcs,granulated_sugar_1kg,wheat_flour_1kg
0,2000-01,13.97,9.30,7.87
1,2000-02,12.25,9.17,7.75
2,2000-03,11.55,8.88,7.65
3,2000-04,13.12,8.94,7.56
4,2000-05,12.76,9.87,7.69


## Average Monthly Salary

In [19]:
avg_monthly_salary = pd.read_excel('data/avg_monthly_nominal_accrued_wages_of_employees_RF_1991-2024.xlsx')

dict_months = {
    "Январь"  : "-01",
    "Февраль" : "-02",
    "Март"    : "-03",
    "Апрель"  : "-04",
    "Май"     : "-05",
    "Июнь"    : "-06",
    "Июль"    : "-07",
    "Август"  : "-08",
    "Сентябрь": "-09",
    "Октябрь" : "-10",
    "Ноябрь"  : "-11",
    "Декабрь" : "-12",
}


frames = []
for column in list(avg_monthly_salary.columns)[1:]:
    tmp = avg_monthly_salary[['Год', column]]
    tmp['Date'] = tmp['Год'].astype(str) + dict_months[column]
    tmp = tmp[['Date', column]]
    tmp.columns = ['date', 'avg_monthly_salary']
    frames.append(tmp)
    
avg_monthly_salary = pd.concat(frames, ignore_index=True)
avg_monthly_salary = avg_monthly_salary.sort_values(by = ['date'], ignore_index=True)

show_stat(avg_monthly_salary)
avg_monthly_salary.head()

Data size: 408 x 2
Min date: 1991-01
Min date: 2024-12

Missing values:
date                  0
avg_monthly_salary    4
dtype: int64


,date,avg_monthly_salary
0,1991-01,0.308
1,1991-02,0.294
2,1991-03,0.337
3,1991-04,0.373
4,1991-05,0.438


## Sanctions

In [20]:
response = requests.get('https://eur-lex.europa.eu/legal-content/EN/TXT/?uri=CELEX%3A02014R0833-20240224')
soup = BeautifulSoup(response.text, 'html.parser') # BeautifulSoup - the library for working with html code

navigation = soup.find('nav', {"class" : "consLegNav"}) # finding dates on website in html code
li_tags = navigation.find_all('li') # finding dates

# creating a df for the dates
dates = []
for i in li_tags:
    dates.append(i.text.strip())
dates = dates[:-1] # deleting the last element

df_sanctions = pd.DataFrame(data = {'date' : dates})
df_sanctions['sanctions'] = 1
df_sanctions['date']=pd.to_datetime(df_sanctions['date']).dt.strftime('%Y-%m')
df_sanctions = df_sanctions.drop_duplicates()

## Basic preprocessing of Inflation Dataset (rename columns + max value per month (if several)) 
df_sanctions = df_sanctions[['date', 'sanctions']]
df_sanctions = df_sanctions.groupby(by = 'date')['sanctions'].max().reset_index() # takes max in case of duplicated months

show_stat(df_sanctions)
df_sanctions.head()

Data size: 22 x 2
Min date: 2014-09
Min date: 2024-12

Missing values:
date         0
sanctions    0
dtype: int64


,date,sanctions
0,2014-09,1
1,2014-12,1
2,2015-10,1
3,2017-12,1
4,2019-07,1


# MAIN DATASET

In [21]:
dates = pd.date_range(start='2011-03-01', end='2023-12-31')
data = pd.DataFrame(data={'date':dates}) 
data['date'] = pd.to_datetime(data['date']).dt.strftime('%Y-%m')
data = data.drop_duplicates().reset_index(drop=True)


#Merging all the features by left join on the column ['date']
data = data.merge(inflation, how='left', on=['date'])
data = data.merge(interest_rate, how='left', on=['date'])
data = data.merge(CPI, how='left', on=['date'])
data = data.merge(unemplyment_rate, how='left', on=['date'])
data = data.merge(gdp, how='left', on=['date'])
data = data.merge(fx_eur_rub, how='left', on=['date'])
data = data.merge(fx_usd_rub, how='left', on=['date'])
data = data.merge(oil_price, how='left', on=['date'])
data = data.merge(gas_price, how='left', on=['date'])
data = data.merge(gold_price, how='left', on=['date'])
data = data.merge(MOEX, how='left', on=['date'])
data = data.merge(exports, how='left', on=['date'])
data = data.merge(imports, how='left', on=['date'])
data = data.merge(external_debt, how='left', on=['date'])
data = data.merge(main_goods, how='left', on=['date'])
data = data.merge(avg_monthly_salary, how='left', on=['date'])
data = data.merge(df_sanctions, how='left', on=['date'])

In [22]:
show_stat(data)
data.head()

Data size: 154 x 20
Min date: 2011-03
Min date: 2023-12

Missing values:
date                      0
inflaton_rate             0
interest_rate             0
CPI                       0
unemplyment_rate          0
GDP                       0
fx_eur_rub                0
fx_usd_rub                0
oil_price                 0
gas_price                 0
gold_price                0
MOEX                     24
exports                   0
imports                   0
external_debt             0
chicken_eggs_10pcs        0
granulated_sugar_1kg      0
wheat_flour_1kg           0
avg_monthly_salary        0
sanctions               137
dtype: int64


,date,inflaton_rate,interest_rate,CPI,unemplyment_rate,GDP,fx_eur_rub,fx_usd_rub,oil_price,gas_price,gold_price,MOEX,exports,imports,external_debt,chicken_eggs_10pcs,granulated_sugar_1kg,wheat_flour_1kg,avg_monthly_salary,sanctions
0,2011-03,9.46,5.25,107.7,7.4,13024.799539,39.635183,28.416026,102.981305,4.069217,1422.617389,NaN,42955.4763,25918.2778,508361.0,39.53,43.31,21.67,22673.0,NaN
1,2011-04,9.61,5.25,108.1,7.0,13024.799539,40.283450,28.076235,110.038501,4.271650,1481.865002,NaN,45211.1719,25941.0420,508361.0,41.98,41.27,21.53,22519.0,NaN
2,2011-05,9.59,8.25,108.7,7.2,13024.799539,39.790900,27.864314,101.356667,4.336143,1511.014288,NaN,43701.5600,26940.5820,508361.0,35.11,39.65,21.34,22779.0,NaN
3,2011-06,9.42,8.25,108.9,6.4,14434.822306,40.085386,27.936823,96.288637,4.516045,1527.995456,NaN,43518.1509,26327.8458,538120.0,30.71,38.66,21.11,24137.0,NaN
4,2011-07,9.01,8.25,108.9,6.1,14434.822306,39.718085,27.854081,97.340500,4.353250,1577.125006,NaN,41487.9474,26029.1594,538120.0,31.05,39.31,21.04,23598.0,NaN


In [23]:
#data.to_csv('data/macro_economics.csv', index=False)